# Predictive Data Modeling 
##### notebook code (C) 2022-2026 Timothy James Becker
## CH4: Classification Overview

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Linear_least_squares_example2.png/330px-Linear_least_squares_example2.png" width=200>

Classification is a general supervised learning problem where we have either qualitative or qualitative column vectors and we wish to predict a qualitative (possibly ordinal) value.  Because this type of prediction will lead to a discrete value (and usually some measure of confidence, probability or likelihood) the methods that we will explore are unique. When we are trying to predict this value which we call its class for a 2-valued qualitative variable we would say we are training a 2-class model.  It is standard in classification methods to reserve a fraction of the observed dataset to use as a model prediction estimate since we do not have the continuous space as we would use with linear regression.  Generally we partition data as either training (using to modify the model parameters) and test (never touches the model and is only used to measure or estimate the accuracy of the model)

### 4.2 Linear Regression Limits

The methods at the end of CH3 showed that we can transform some quantitative variables using a binary coding scheme and a piece-wise function to assign the 0 and 1 (dummy variable technique):

$
x_i =
  \begin{cases}
    1  & \quad \text{if the }i\text{th value is }=A\\
    0  & \quad \text{if the }i\text{th value is }=B
  \end{cases}
$

The main limitation here is binary classification as oppose to the general classification problem where we have three or more values we wish to predict. When we have more than two variables it remains problematic to create piece-wise assignments since the distance between what has been assigned and what is actually being observe may vary greatly. Lets look at a three-class dummy variable coding that will not provide a good linear regression result:

$
x_i =
  \begin{cases}
    0  & \quad \text{if the }i\text{th value is }=A\\
    1  & \quad \text{if the }i\text{th value is }=B\\
    2  & \quad \text{if the }i\text{th value is }=Z
  \end{cases}
$

This result will not be great when the distances coded do not hold in the transformed space.  Look at the $A$ to $B$ space and assume we are simply using the lexicographic ordering from English here.  $A$ is adjacent to $B$ or $(1-0)=1$ away.  But what about $B$ and $Z \rightarrow (2-1)=1$?

Fine you may say, we can use the distance from the lexicographical ordering then:

$
x_i =
  \begin{cases}
    0  & \quad \text{if the }i\text{th value is }=A\\
    1  & \quad \text{if the }i\text{th value is }=B\\
    25  & \quad \text{if the }i\text{th value is }=Z
  \end{cases}
$

The issue here is that it may work in some data domains (ordinal) but not in the general case (non-ordinal):

$
x_i =
  \begin{cases}
    0  & \quad \text{if the }i\text{th value is }=\text{Male}\\
    1  & \quad \text{if the }i\text{th value is }=\text{Female}\\
    2  & \quad \text{if the }i\text{th value is }=\text{Other}
  \end{cases}
$

### 4.3 Logistic Regression

Here we will start with a very simply method that uses probability theory.  Instead of trying to predict directly the qualitative value (like linear regression would do) we simply try to predict the probability or chance that our observed column vectors would also have either the 0 or 1 class in the test data given observations from the training data.

More formally we express the probability of a random variable $X$ having value $1$ as: $Pr(X=1)=0.1$.  In this example there was a $0.1$ probability. For linear regression we are concerned with predicting our variable Y using one of the column vectors:

$Pr(Y=1 | X)$ is a conditional probability that is described as: The probability of $Y$ having value $1$ given the value for $X$.

Now we adapt the normal linear regression to use the conditional probability and provide a method to bound the result to the valid probability range of $[0,1]$:

$p(X)= \beta_0 + \beta_1{X}$ and turn it into logistic regression as:

$p(X)=\frac{e^{\beta_0 + \beta_1{X}}}{1+e^{\beta_0 + \beta_1{X}}}$

we can then manipulate it to become the logit as:

$\displaystyle \text{log}\left( \frac{p(x)}{1-p(X)} \right) = \beta_0 + \beta_1{X}$

<u>Maximum Likelihood</u>

Next we will use a more robust method (one that is adapted for probabilities!) than minimum sum of squares called maximum lilkelihood:

$\displaystyle{\cal L} (\beta_0,\beta_1)=\prod_{i:y_i=1}p(X_i) \prod_{i':y_i'=0}(1-p(X_i))$

This will work to fit non-linear models...

<u>Prediction</u>

Now once we have a fit using the likelihood estimates for $\beta_0,\beta_1$ we can plug them in as our estimate:

$\hat{p}(X)= \frac{e^{\hat{\beta}_0 + \hat{\beta}_1{X}}}{1+e^{\hat{\beta}_0 + \hat{\beta}_1{X}}}$

#### <u>Multiple Logistic Regression</u>

Now we can apply more variables just as in linear regression:

$p(X)=\frac{e^{\beta_0 + \beta_1{X_1} + ... + \beta_p{X_p}}}{1+e^{\beta_0 + \beta_1{X_1}+ ...+\beta_p{X_p}}}$


$\displaystyle \text{log}\left( \frac{p(X)}{1-p(X)} \right) = \beta_0 + \beta_1{X_1} + ... + \beta_p{X_p}$ 

In [306]:
#lets take a look at the Default dataset which has a binary value we want to predict called default
import numpy as np
import pandas as pd
from ISLP import load_data
data = load_data('Default')
data[:10]

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
5,No,Yes,919.588530,7491.558572
6,No,No,825.513331,24905.226578
7,No,Yes,808.667504,17600.451344
8,No,No,1161.057854,37468.529288
9,No,No,0.000000,29275.268293


In [307]:
default = np.asarray(data[:]['default'].cat.codes,dtype=np.float32)
student = np.asarray(data[:]['student'].cat.codes,dtype=np.float32)
cols = list(set(data.columns).difference(set(['default','student'])))
data = pd.DataFrame(data[:][cols],columns=cols)
data.loc[:,'default'] = default
data.loc[:,'student'] = student
data[:10]

,balance,income,default,student
0,729.526495,44361.625074,0.0,0.0
1,817.180407,12106.134700,0.0,1.0
2,1073.549164,31767.138947,0.0,0.0
3,529.250605,35704.493935,0.0,0.0
4,785.655883,38463.495879,0.0,0.0
5,919.588530,7491.558572,0.0,1.0
6,825.513331,24905.226578,0.0,0.0
7,808.667504,17600.451344,0.0,1.0
8,1161.057854,37468.529288,0.0,0.0
9,0.000000,29275.268293,0.0,0.0


In [308]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import TweedieRegressor
Lin = LinearRegression().fit(data[:][['income']],data[:][['default']])
Lin

LinearRegression()

In [176]:
np.max(Lin.predict(data[:][['income']]))

0.04205423029518606

Using the logistic regression from scikitlearn you can read about [here](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

In [199]:
#Next pass it to the logistic regression
Log  = LogisticRegression(C=1.0,solver="liblinear").fit(data[:][['income']],data[:]['default'])

Now we will predict a value and see how it would work out

In [200]:
pred = Log.predict_proba(data[:][['income']]) #get the probabilties for class 0 and class 1, use Log.predict to get the np.argmax(probs)
pred

array([[0.99237929, 0.00762071],
       [0.79064171, 0.20935829],
       [0.97031088, 0.02968912],
       ...,
       [0.99839983, 0.00160017],
       [0.98244815, 0.01755185],
       [0.86423477, 0.13576523]])

And we will check overall accuracy (how many times the data was Default=1 and the Default prediction=1)

In [204]:
p = np.zeros((len(data)),dtype=np.float32)
p[np.where(pred[:,1]>=0.5)] = 1.0
np.sum(data[:]['default']*p)/len(data)

0.0

Not great!, play around with the selection point in the where statement to calibrate the model for maximal accuracy

#### <u>Partitioning Data for Train and Test</u>

The above outcome should make sense but we gave our predictor the answers!  Imagine if you gave your friend the answer to a difficult Math problem ahead and then were surprised when they guessed the answer... It is very similar to LLMs since they have already been given your text book and are usually looking up your questions!  What about if you gave your model a new unseen problem and they got the right answer… More impressive, right?

We will partition two different ways, one is a standard method: randomly partition but it will often not be a good estimator of the model performance because you are not ensuring the answers have not been seen! The second way ensures (mathematically) that the new values given to the predictor have never been seen.

#### [1] Random Partitioning

In [268]:
#random partitioning
split  = 0.1
train  = list(np.random.choice(range(len(data)),int(len(data)*(1.0-split)),replace=False))
test   = list(set(range(len(data))).difference(set(train)))
len(train),len(test)

(9000, 1000)

In [269]:
Log  = LogisticRegression(C=1.0,solver="liblinear").fit(data.loc[train][['income']],data.loc[train]['default']) #build on the trainin gpartition
pred = Log.predict_proba(data.loc[test][['income']])                                                            #predict on the test partition

p = np.zeros((len(test)),dtype=np.float32)
p[np.where(pred[:,1]>=0.01)] = 1.0
np.sum(data.loc[test]['default']*p)/len(test)

0.02

Basically the same performance as above since we have not ensured that the values we have provided are not seen...

#### [2] Domain Partitioning (ensure values have not been seen)

In [270]:
p_point = 50766
train   = list(np.where(data['income']<=p_point)[0])
test    = list(np.where(data['income']>p_point)[0])
len(train),len(test)

(9000, 1000)

In [271]:
Log  = LogisticRegression(C=1.0,solver="liblinear").fit(data.loc[train][['income']],data.loc[train]['default']) #build on the trainin gpartition
pred = Log.predict_proba(data.loc[test][['income']])                                                            #predict on the test partition

p = np.zeros((len(test)),dtype=np.float32)
p[np.where(pred[:,1]>=0.01)] = 1.0
np.sum(data.loc[test]['default']*p)/len(test)

0.0

Basically this is a good lowerbound on the model performance and the random is the upperbound!

#### [3] Random Domain Partitioning (combining both methods)

##### (a) build the distribution to sample from (we take pairs of values from the X,Y)

In [274]:
D = {}
for i in range(len(data)):
    income,default = data.loc[i]['income'],data.loc[i]['default'] #get the X,Y values here
    k = (income,default)                                          #make a key using a tuple of individual values
    if k in D: D[k] += [i]                                        #keep the row index for any duplicates
    else:      D[k]  = [i]                                        #because these are unique they can be randomly partitioned by key

(b) now sample the keys randomly (the values have not been seen in the partitions now)

In [283]:
split    = 0.1
ks       = list(D)
train_i  = list(np.random.choice(range(len(ks)),int(len(ks)*(1.0-split)),replace=False)) #sample from the unique keys of D
test_i   = list(set(range(len(ks))).difference(set(train_i)))                            #then we use these to pack up the keys

train,test = [],[]            #now we unpack the instances of those keys
for i in range(len(ks)):      #in case of duplicates they will all go
    if i in train_i:          #into the same partition->train or test
        train += D[ks[i]]
    else:
        test  += D[ks[i]]
len(train),len(test)

(9000, 1000)

In [284]:
Log  = LogisticRegression(C=1.0,solver="liblinear").fit(data.loc[train][['income']],data.loc[train]['default']) #build on the trainin gpartition
pred = Log.predict_proba(data.loc[test][['income']])                                                            #predict on the test partition

p = np.zeros((len(test)),dtype=np.float32)
p[np.where(pred[:,1]>=0.01)] = 1.0
np.sum(data.loc[test]['default']*p)/len(test)

0.022

This is now in between the upper bound and lower bound. When there are many duplicate keys this will be ~ lower bound, when there are no duplicate keys it will ~ be the upper bound

In [295]:
#now an industrial strength version that can take multiple Xs and multiple Ys into account...
def partition_by_key(data,xs,ys,split=0.1):
    D = {}
    for i in range(len(data)):
        k = tuple([])
        for x in xs: k += (data.loc[i][x],)   #gather all the column names
        for y in ys: k += (data.loc[i][y],)   #and make a key
        if k in D: D[k] += [i]   #keep the row index for any duplicates
        else:      D[k]  = [i]   #because these are unique they can be randomly partitioned by key
    ks       = list(D)
    train_i  = list(np.random.choice(range(len(ks)),int(len(ks)*(1.0-split)),replace=False)) #sample from the unique keys of D
    test_i   = list(set(range(len(ks))).difference(set(train_i)))                            #then we use these to pack up the keys
    
    train,test = [],[]            #now we unpack the instances of those keys
    for i in range(len(ks)):      #in case of duplicates they will all go
        if i in train_i:          #into the same partition->train or test
            train += D[ks[i]]
        else:
            test  += D[ks[i]]
    return train,test

In [296]:
train,test = partition_by_key(data,['income','balance'],['default'])
len(train),len(test)

(9000, 1000)

#### Multinomial Logistic Regression

Now we will extend the 2-class to some larger number K-class using the softmax encoding:

for $k = 1 $ to $ K$:

$\quad Pr(Y=k|X=x) = \displaystyle\frac{e^{\beta_{k0}+\beta_{k1}x_1 + ... + \beta_{kp}x_p}}{\sum_{l=1}^{K}e^{\beta_{l0}+\beta_{l1}x_1 + ... + \beta_{lp}x_p}} \implies$ 

$\quad \text{log}\left( \frac{Pr(Y=k|X=x)}{Pr(Y=k'|X=x)} \right) = (\beta_{k1}-\beta_{k'0})+(\beta_{k1}-\beta_{k'1})x_1 + ... + (\beta_{kp}-\beta_{k'p})x_p$



In [297]:
xs   = ['income','balance']               #modify this list to build multiple-logistic regression
ys   = ['default']                        #here is your [0-1] variable you want to predict
train,test = partition_by_key(data,xs,ys)

Log  = LogisticRegression(C=1.0,solver="liblinear").fit(data.loc[train][xs],data.loc[train][ys[0]]) #build on the training partition
pred = Log.predict_proba(data.loc[test][xs])                                                        #predict on the test partition

p = np.zeros((len(test)),dtype=np.float32)
p[np.where(pred[:,1]>=0.01)] = 1.0
np.sum(data.loc[test][ys[0]]*p)/len(test)

0.021

### 4.6 General Linear Models (GLM)

We are going to look at another example of when not to use linear regression to model data: When the underlying column vector is counts or positive integer data. Here like in the qualitative domain the linear regression can yeild non-sensical results like negative counts (which don't exist since 0 is the smallest a count could be). scikit-learn has some nice details about the impementation [here](https://scikit-learn.org/stable/modules/linear_model.html#generalized-linear-models)

Next we will look at a probabilty distribution that is designed for counts.  This is just like the binomial and multinomial (from logistic regression) were designed to deal with the descrete prediction values in the qualitative variables.

<u>Poisson Distribution (AKA waiting...)</u>

We let random varible $Y \in \{1,2,3,...\}$ represent counts and so it follows the Poisson distribution if:

$Pr(Y=k) = \frac{e^{-\lambda}\lambda^k}{k!} \text{ for }k=0,1,2,...$

$\lambda = E(Y) = Var(Y)$

We can then use this counting probabilty to estimate count column vectors like bikers:

$\lambda = E(Y) \;\text{~} \lambda(X_1,X_2,...,X_p) \implies \text{log}(\lambda(X_1,X_2,...,X_p)) = \beta_0+\beta_1X_1+...+\beta_pX_p$

This leads to our maximum likelihood estimate as:

$\displaystyle{\cal L}(\beta_0,\beta_1,...,\beta_p) = \prod_{i=1}^{n}\frac{e^{-\lambda(x_i)} \lambda(x_i^{y_i}) }{y_i!}$

In [744]:
data = load_data('Bikeshare')
data

,season,mnth,day,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,bikers
0,1,Jan,1,0,0,6,0,clear,0.24,0.2879,0.81,0.0000,3,13,16
1,1,Jan,1,1,0,6,0,clear,0.22,0.2727,0.80,0.0000,8,32,40
2,1,Jan,1,2,0,6,0,clear,0.22,0.2727,0.80,0.0000,5,27,32
3,1,Jan,1,3,0,6,0,clear,0.24,0.2879,0.75,0.0000,3,10,13
4,1,Jan,1,4,0,6,0,clear,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8640,1,Dec,365,19,0,6,0,clear,0.42,0.4242,0.54,0.2239,19,73,92
8641,1,Dec,365,20,0,6,0,clear,0.42,0.4242,0.54,0.2239,8,63,71
8642,1,Dec,365,21,0,6,0,clear,0.40,0.4091,0.58,0.1940,2,50,52
8643,1,Dec,365,22,0,6,0,clear,0.38,0.3939,0.62,0.1343,2,36,38


In [829]:
xs    = ['windspeed']               
ys    = ['bikers']
train,test = partition_by_key(data,xs,ys)

train_data = data.loc[train]
test_data  = data.loc[test]
len(train_data),len(test_data)

(7831, 814)

In [830]:
Lin = LinearRegression().fit(train_data[xs],train_data[ys])
Lin

LinearRegression()

In [831]:
Log  = LogisticRegression(solver='liblinear').fit(train_data[xs],train_data[ys[0]])
Log

LogisticRegression(solver='liblinear')

In [847]:
Glm = TweedieRegressor(power=1,alpha=0.5).fit(train_data[xs],train_data[ys[0]]) #power=1 and link='log' is the Poisson distribution/regression
Glm

TweedieRegressor(alpha=0.5, power=1)

In [852]:
def mean_power_error(y_true,y_pred,power=1):
    diff   = np.sum(np.abs(y_true-y_pred**power),axis=0)/len(y_true)
    return diff

In [853]:
pred_lin = Lin.predict(test_data[xs]) 
mean_power_error(test_data[ys],pred_lin)

bikers    104.857328
dtype: float64

In [854]:
pred_log = Log.predict(test_data[xs]).reshape(len(test_data),1)
mean_power_error(test_data[ys],pred_log)

bikers    141.732187
dtype: float64

In [855]:
pred_glm = Glm.predict(test_data[xs]).reshape(len(test_data),1)
mean_power_error(test_data[ys],pred_glm)

bikers    104.850171
dtype: float64

In [856]:
test_data.loc[:,'Log'] = pred_log
test_data.loc[:,'Lin'] = pred_lin
test_data.loc[:,'Glm'] = pred_glm
test_data

,season,mnth,day,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,bikers,Log,Lin,Glm
18,1,Jan,1,18,0,6,0,light rain/snow,0.42,0.4242,0.88,0.2537,9,26,35,5,149.440802,148.059536
521,1,Jan,23,20,0,0,0,clear,0.10,0.1061,0.36,0.2537,4,31,35,5,149.440802,148.059536
44,1,Jan,2,21,0,0,0,clear,0.26,0.2273,0.44,0.3284,11,20,31,1,156.341125,153.882942
454,1,Jan,20,23,0,4,1,cloudy/misty,0.24,0.2121,0.65,0.3284,3,28,31,1,156.341125,153.882942
7130,4,Oct,302,17,0,6,0,light rain/snow,0.22,0.1970,0.93,0.3284,3,28,31,1,156.341125,153.882942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8540,1,Dec,361,14,0,2,1,light rain/snow,0.42,0.4242,0.82,0.3881,0,14,14,1,161.855840,158.701234
8546,1,Dec,361,20,0,2,1,clear,0.32,0.3333,0.93,0.0896,3,66,69,5,134.282263,136.028903
8613,1,Dec,364,16,0,5,1,clear,0.42,0.4242,0.47,0.1940,36,247,283,5,143.926087,143.564334
8633,1,Dec,365,12,0,6,0,clear,0.52,0.5000,0.39,0.2985,93,180,273,3,153.579148,151.525022


In [857]:
#now we can go back and tackle NHANES with our three tools: linear:numeric, logistic:qualitative, poisson:counts
data   = pd.read_csv('NHANES.csv')
data

,SurveyYr,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,...,AgeFirstMarij,RegularMarij,AgeRegMarij,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation
0,2009_10,55829,female,28,20-29,343.0,White,NaN,CollegeGrad,Married,...,15.0,No,NaN,Yes,Yes,13.0,20.0,1.0,No,Heterosexual
1,2009_10,57112,male,14,10-19,170.0,White,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009_10,60232,male,80,NaN,NaN,White,NaN,8thGrade,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009_10,59919,male,22,20-29,268.0,White,NaN,HighSchool,NeverMarried,...,10.0,Yes,10.0,Yes,Yes,18.0,3.0,1.0,No,Heterosexual
4,2009_10,56351,male,1,0-9,16.0,White,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2011_12,64065,male,46,40-49,NaN,White,White,SomeCollege,Married,...,NaN,No,NaN,No,Yes,17.0,7.0,1.0,No,Heterosexual
9996,2011_12,66189,male,47,40-49,NaN,Mexican,Mexican,9_11thGrade,Married,...,15.0,Yes,20.0,No,Yes,18.0,7.0,1.0,No,Heterosexual
9997,2011_12,63714,male,48,40-49,NaN,White,White,SomeCollege,Married,...,16.0,No,NaN,No,Yes,17.0,4.0,1.0,No,Heterosexual
9998,2011_12,68501,male,52,50-59,NaN,White,White,SomeCollege,Married,...,16.0,Yes,16.0,Yes,Yes,16.0,50.0,1.0,No,Heterosexual


### <u>Exercises</u>
#### [1] explore some of the other variables in the Bikers dataset and apply the three model types to three appropriate response variables
#### [2] look at the results of the predicted values for the Bikers modles (as opposed to just using a since metric like mean_power_error and evaluate the results
#### [3] Look over the NHANES table above and describe which linear model type is a good choice for each
#### [4] Using past experience from working with NHANES try building a multi-linear regression model to predict a: catagorical variable, a positive variable, a numeric variable